In [115]:
import pandas as pd
import numpy as np
import datetime
week_end_saturday=datetime.date(year=2018,month=11,day=3)

# Added Google SOTF account

# Google PLA

In [116]:
Google_PLA_1=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Google_Adwords/Shopping/MetroArea_Shopping_Google_Oct2016-Sep2017_CSV.csv",skiprows=2,dtype=str)
Google_PLA_2=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Google_Adwords/Shopping/MetroArea_Shopping_Google_Oct2017-20181213_CSV.csv",skiprows=2,dtype=str)

del Google_PLA_1['Currency']
del Google_PLA_2['Currency']

Google_PLA_1['Day']=Google_PLA_1['Day'].apply(lambda x: datetime.datetime.strptime(x, '%b %d, %Y').date())
Google_PLA_2['Day']=Google_PLA_2['Day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())


Google_PLA_2.columns=Google_PLA_1.columns
Google_PLA=Google_PLA_1.append(Google_PLA_2)
# Keep US Only
# print(Google_PLA['Country/Territory'].unique())

Google_PLA=Google_PLA[Google_PLA['Country/Territory']=='United States']
Google_PLA.reset_index(inplace=True)
del Google_PLA['index']
print(Google_PLA['Campaign type'].unique())

['Shopping']


In [117]:
def clean_Google_num(df):
    if 'Impressions' in df.columns:
        df['Impressions']=df['Impressions'].apply(lambda x: x.replace(",",""))
        df['Impressions']=df['Impressions'].astype(int)
    if 'Clicks' in df.columns:
        df['Clicks']=df['Clicks'].apply(lambda x: x.replace(",",""))
        df['Clicks']=df['Clicks'].astype(int)
    if 'Cost' in df.columns:
        df['Cost']=df['Cost'].apply(lambda x: x.replace(",",""))
        df['Cost']=df['Cost'].apply(lambda x: x.replace("$",""))
        df['Cost']=df['Cost'].astype(float)
    return df

In [118]:
Google_PLA=Google_PLA[['Day','Metro area','Impressions','Clicks','Cost']]
Google_PLA=clean_Google_num(Google_PLA)

Google_PLA['weekday']=Google_PLA['Day'].apply(lambda x: x.weekday())
Google_PLA=Google_PLA[(Google_PLA['Day']>=datetime.date(year=2016,month=10,day=2)) & (Google_PLA['Day']<=week_end_saturday)]
Google_PLA.reset_index(inplace=True)
del Google_PLA['index']

In [119]:
Google_PLA['week_start_date']=np.where(Google_PLA['weekday']==6,
                                          Google_PLA['Day'],
                                          Google_PLA['Day'].apply(lambda x: x-datetime.timedelta(days=(x.weekday()+1))))
Google_PLA['week_end_date']=Google_PLA['week_start_date']+datetime.timedelta(days=6)
Google_PLA=Google_PLA.groupby(['week_start_date','week_end_date','Metro area'])['Impressions','Clicks','Cost'].sum()
Google_PLA.reset_index(inplace=True)
Google_PLA['media']="Digital"
Google_PLA['submedia']="PLA"
Google_PLA['placement']="Google"
Google_PLA=Google_PLA[['week_start_date','week_end_date','Metro area','media','submedia','placement','Impressions','Clicks','Cost']]
Google_PLA.columns=['week_start_date','week_end_date','DMA','media','submedia','placement','Impressions','Clicks','Cost']

Google_PLA.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Aggregated_PLA_Google_Till_"+str(week_end_saturday)+".csv",index=False)

In [120]:
Google_SEM_GDN_1=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Google_Adwords/Text_Ads/MetroArea_CampaignType_SEM_Oct2016-Sep2017_CSV.csv",skiprows=2,dtype=str)
del Google_SEM_GDN_1['Currency']
Google_SEM_GDN_2=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Google_Adwords/Text_Ads/MetroArea_CampaignType_SEM_Oct2017-20181201_CSV.csv",skiprows=2,dtype=str)
del Google_SEM_GDN_2['Currency']

Google_SEM_GDN_1['Day']=Google_SEM_GDN_1['Day'].apply(lambda x: datetime.datetime.strptime(x, '%b %d, %Y').date())
Google_SEM_GDN_2['Day']=Google_SEM_GDN_2['Day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Google_SEM_GDN_2.columns=Google_SEM_GDN_1.columns.tolist()
print(Google_SEM_GDN_1['Campaign type'].unique())
print(Google_SEM_GDN_2['Campaign type'].unique())

Google_SOTF_account=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Google_Adwords/SOTF/MetroArea_CampaignType_SOTF_Jan2018-20181201_CSV.csv",skiprows=2,dtype=str)
del Google_SOTF_account['Currency']
Google_SOTF_account['Day']=Google_SOTF_account['Day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Google_SOTF_account=Google_SOTF_account[['Day', 'Metro area (Geographic)', 'Country/Territory (Geographic)', 'Campaign type', 'Clicks', 'Impressions','Cost']]

Google_SOTF_account.columns=Google_SEM_GDN_1.columns
print(Google_SOTF_account['Campaign type'].unique())

['Search Only' 'Display Only']
['Search Only' 'Display Only']
['Search Only']


In [121]:
Google_SEM_GDN=Google_SEM_GDN_1.append(Google_SEM_GDN_2).append(Google_SOTF_account)

# Keep US only, no missing found
# print(Google_SEM_GDN['Country/Territory'].unique())
Google_SEM_GDN=Google_SEM_GDN[Google_SEM_GDN['Country/Territory']=='United States']
Google_SEM_GDN.reset_index(inplace=True)
del Google_SEM_GDN['index']
Google_SEM_GDN['Impressions'].apply(lambda x: x.replace(",","")).astype(float).sum()


338225648.0

# Google SEM

In [122]:
Google_SEM=Google_SEM_GDN[Google_SEM_GDN['Campaign type']=='Search Only']
del Google_SEM['Campaign type']
Google_SEM=clean_Google_num(Google_SEM)

Google_SEM['weekday']=Google_SEM['Day'].apply(lambda x: x.weekday())
Google_SEM=Google_SEM[(Google_SEM['Day']>=datetime.date(year=2016,month=10,day=2)) & (Google_SEM['Day']<=week_end_saturday)]
Google_SEM.reset_index(inplace=True)
del Google_SEM['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [123]:
Google_SEM['week_start_date']=np.where(Google_SEM['weekday']==6,
                                          Google_SEM['Day'],
                                          Google_SEM['Day'].apply(lambda x: x-datetime.timedelta(days=(x.weekday()+1))))
Google_SEM['week_end_date']=Google_SEM['week_start_date']+datetime.timedelta(days=6)
Google_SEM=Google_SEM.groupby(['week_start_date','week_end_date','Metro area'])['Impressions','Clicks','Cost'].sum()
Google_SEM.reset_index(inplace=True)
Google_SEM['media']="Digital"
Google_SEM['submedia']="SEM"
Google_SEM['placement']="Google"
Google_SEM=Google_SEM[['week_start_date','week_end_date','Metro area','media','submedia','placement','Impressions','Clicks','Cost']]
Google_SEM.columns=['week_start_date','week_end_date','DMA','media','submedia','placement','Impressions','Clicks','Cost']

Google_SEM.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Aggregated_SEM_Google_Till_"+str(week_end_saturday)+".csv",index=False)

In [124]:
Google_SEM['week_end_date'].max()

datetime.date(2018, 11, 3)

# Google GDN

In [125]:
Google_GDN=Google_SEM_GDN[Google_SEM_GDN['Campaign type']=='Display Only']
del Google_GDN['Campaign type']
Google_GDN=clean_Google_num(Google_GDN)

Google_GDN['weekday']=Google_GDN['Day'].apply(lambda x: x.weekday())
Google_GDN=Google_GDN[(Google_GDN['Day']>=datetime.date(year=2016,month=10,day=2)) & (Google_GDN['Day']<=week_end_saturday)]
Google_GDN.reset_index(inplace=True)
del Google_GDN['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [126]:
Google_GDN['week_start_date']=np.where(Google_GDN['weekday']==6,
                                          Google_GDN['Day'],
                                          Google_GDN['Day'].apply(lambda x: x-datetime.timedelta(days=(x.weekday()+1))))
Google_GDN['week_end_date']=Google_GDN['week_start_date']+datetime.timedelta(days=6)
Google_GDN=Google_GDN.groupby(['week_start_date','week_end_date','Metro area'])['Impressions','Clicks','Cost'].sum()
Google_GDN.reset_index(inplace=True)
Google_GDN['media']="Digital"
Google_GDN['submedia']="Programmatic"
Google_GDN['placement']="GDN"
Google_GDN=Google_GDN[['week_start_date','week_end_date','Metro area','media','submedia','placement','Impressions','Clicks','Cost']]
Google_GDN.columns=['week_start_date','week_end_date','DMA','media','submedia','placement','Impressions','Clicks','Cost']

Google_GDN.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Aggregated_GDN_Google_Till_"+str(week_end_saturday)+".csv",index=False)

# Bing PLA

In [127]:
Bing_PLA=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Bing_Ads/Using_MetroArea_Shopping_Bing_20161001_20181031.csv",skiprows=9,skipfooter=2)
Bing_PLA=Bing_PLA[['Gregorian date','Country/region','Metro area','Impressions','Clicks','Spend']]
Bing_PLA.columns=['Day','Country','DMA','Impressions','Clicks','Cost']
Bing_PLA['DMA']=Bing_PLA['DMA'].fillna("xx")
# US only in Bing Shopping
Bing_PLA['Day']=Bing_PLA['Day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Bing_PLA['weekday']=Bing_PLA['Day'].apply(lambda x: x.weekday())
Bing_PLA=Bing_PLA[(Bing_PLA['Day']>=datetime.date(year=2016,month=10,day=2)) & (Bing_PLA['Day']<=week_end_saturday)]
Bing_PLA.reset_index(inplace=True)
del Bing_PLA['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [128]:
Bing_PLA['week_start_date']=np.where(Bing_PLA['weekday']==6,
                                          Bing_PLA['Day'],
                                          Bing_PLA['Day'].apply(lambda x: x-datetime.timedelta(days=(x.weekday()+1))))
Bing_PLA['week_end_date']=Bing_PLA['week_start_date']+datetime.timedelta(days=6)
Bing_PLA=Bing_PLA.groupby(['week_start_date','week_end_date','DMA'])['Impressions','Clicks','Cost'].sum()
Bing_PLA.reset_index(inplace=True)
Bing_PLA['media']="Digital"
Bing_PLA['submedia']="PLA"
Bing_PLA['placement']="Bing"
Bing_PLA=Bing_PLA[['week_start_date','week_end_date','DMA','media','submedia','placement','Impressions','Clicks','Cost']]

Bing_PLA.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Aggregated_PLA_Bing_Till_"+str(week_end_saturday)+".csv",index=False)



# Bing SEM

In [129]:
Bing_SEM=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Bing_Ads/Using_MetroArea_SEM_Bing_20161001_20181031.csv",skiprows=9,skipfooter=2)
Bing_SEM=Bing_SEM[['Gregorian date','Country/region','Metro area','Impressions','Clicks','Spend']]
Bing_SEM.columns=['Day','Country','DMA','Impressions','Clicks','Cost']
Bing_SEM=Bing_SEM[Bing_SEM['Country'].isin(['United States',np.nan])]

Bing_SEM['Country'].fillna("xx",inplace=True)
Bing_SEM['DMA'].fillna("xx",inplace=True)

Bing_SEM['Day']=Bing_SEM['Day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Bing_SEM['weekday']=Bing_SEM['Day'].apply(lambda x: x.weekday())
Bing_SEM=Bing_SEM[(Bing_SEM['Day']>=datetime.date(year=2016,month=10,day=2)) & (Bing_SEM['Day']<=week_end_saturday)]
Bing_SEM.reset_index(inplace=True)
del Bing_SEM['index']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [130]:
Bing_SEM['week_start_date']=np.where(Bing_SEM['weekday']==6,
                                          Bing_SEM['Day'],
                                          Bing_SEM['Day'].apply(lambda x: x-datetime.timedelta(days=(x.weekday()+1))))
Bing_SEM['week_end_date']=Bing_SEM['week_start_date']+datetime.timedelta(days=6)
Bing_SEM=Bing_SEM.groupby(['week_start_date','week_end_date','DMA'])['Impressions','Clicks','Cost'].sum()
Bing_SEM.reset_index(inplace=True)
Bing_SEM['media']="Digital"
Bing_SEM['submedia']="SEM"
Bing_SEM['placement']="Bing"
Bing_SEM=Bing_SEM[['week_start_date','week_end_date','DMA','media','submedia','placement','Impressions','Clicks','Cost']]

Bing_SEM.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/Aggregated_SEM_Bing_Till_"+str(week_end_saturday)+".csv",index=False)

# Criteo

In [131]:
# No need any more since it's already in datorama

'''
Criteo_by_Day=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull March 23/Criteo/Extract_20161001_20180228.csv",sep=";")
Criteo_by_Day['Day']=Criteo_by_Day['Day'].apply(lambda x: datetime.datetime.strptime(x,"%a %m/%d/%Y").date())
Criteo_by_Day=Criteo_by_Day[(Criteo_by_Day['Day']>=datetime.datetime.strptime('2016-10-02',"%Y-%m-%d").date()) & (Criteo_by_Day['Day']<=datetime.datetime.strptime('2018-02-27',"%Y-%m-%d").date())]
Criteo_by_Day['weekday']=Criteo_by_Day['Day'].apply(lambda x:x.weekday())
Criteo_by_Day=Criteo_by_Day.reset_index()
del Criteo_by_Day['index']
Criteo_by_Day['week_start_date']=np.where(Criteo_by_Day['weekday']==6,
                                          Criteo_by_Day['Day'],
                                         Criteo_by_Day['Day'].apply(lambda x:x-datetime.timedelta(days=(x.weekday()+1))))
Criteo_by_Day['week_end_date']=Criteo_by_Day['week_start_date']+datetime.timedelta(days=6)
Criteo_by_Day['media']="Digital"
Criteo_by_Day['submedia']="Programmatic"
Criteo_by_Day['placement']="Display Retargeting"
Criteo_by_Day['cleaned dma']="National"
Criteo_by_Day=Criteo_by_Day[['week_start_date','week_end_date','cleaned dma','media','submedia','placement','Impressions','Clicks','Cost']]
Criteo_by_Day=Criteo_by_Day.groupby(['week_start_date','week_end_date','cleaned dma','media','submedia','placement'])['Impressions','Clicks','Cost'].sum()
Criteo_by_Day.rename(columns={'Impressions':'impression','Clicks':'click','Cost':'cost'}, inplace=True)
Criteo_by_Day.reset_index(inplace=True)

Criteo_by_Day.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Aug_4/Criteo_Till_20180227.csv",index=False)

'''

'\nCriteo_by_Day=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull March 23/Criteo/Extract_20161001_20180228.csv",sep=";")\nCriteo_by_Day[\'Day\']=Criteo_by_Day[\'Day\'].apply(lambda x: datetime.datetime.strptime(x,"%a %m/%d/%Y").date())\nCriteo_by_Day=Criteo_by_Day[(Criteo_by_Day[\'Day\']>=datetime.datetime.strptime(\'2016-10-02\',"%Y-%m-%d").date()) & (Criteo_by_Day[\'Day\']<=datetime.datetime.strptime(\'2018-02-27\',"%Y-%m-%d").date())]\nCriteo_by_Day[\'weekday\']=Criteo_by_Day[\'Day\'].apply(lambda x:x.weekday())\nCriteo_by_Day=Criteo_by_Day.reset_index()\ndel Criteo_by_Day[\'index\']\nCriteo_by_Day[\'week_start_date\']=np.where(Criteo_by_Day[\'weekday\']==6,\n                                          Criteo_by_Day[\'Day\'],\n                                         Criteo_by_Day[\'Day\'].apply(lambda x:x-datetime.timedelta(days=(x.weekday()+1))))\nCriteo_by_Day[\'week_end_date\']=Criteo_by_Day[\'week_start_date\']+datetime.timedelta(days=6)\nCriteo_by_Day[\'medi

# Jian_Overall_TMR_Update

In [132]:
output=Google_PLA.append(Google_SEM).append(Google_GDN).append(Bing_SEM).append(Bing_PLA)
output=output.rename(columns={"DMA":"dma","Impressions":"impression","Clicks":"click","Cost":"cost"})

In [133]:
DMA_cleaning=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/DMA cleaning.xlsx")
output=pd.merge(output,DMA_cleaning,on="dma",how="left")
output['cleaned dma']=output['cleaned dma'].replace("xx","National")
output[pd.isnull(output['cleaned dma'])]

# Check the small number

,week_start_date,week_end_date,dma,media,submedia,placement,impression,click,cost,cleaned dma
79873,2018-04-29,2018-05-05,Ottawa,Digital,PLA,Bing,1,0,0.0,NaN
80052,2018-05-06,2018-05-12,London,Digital,PLA,Bing,3,0,0.0,NaN
80081,2018-05-06,2018-05-12,Ottawa,Digital,PLA,Bing,1,0,0.0,NaN
80490,2018-05-20,2018-05-26,Newcastle upon Tyne,Digital,PLA,Bing,2,0,0.0,NaN


In [134]:
output=output[~pd.isnull(output['cleaned dma'])]

output.reset_index(inplace=True)
del output['index']
output.to_csv("/home/jian/Projects/Big_Lots/TMR/Jian_TMR_Update/Repull_Dec_13/BL_SEM_PLA_GDN_TMR_Till_"+str(week_end_saturday)+"_JL.csv",index=False)